# Perfect foresight experiments with dolo

** Spencer Lyon **

** Pablo Winant **

January 2018

In this notebook we will explore the effects of techonology and fiscal policy shocks in a nonstochastic model. The model and figures are inspired by chapter 11 of Recursive Macroeconomic Theory 3rd edition (RMT3) by Ljungqvist and Sargent. 

We will focus on the computational aspects of the exercises and will refer the interested reader to section 11.9 of RMT3 for a discussion on the underlying economics.

## Model

A representative household has preferences over consumption that are ordered by 

$$U := \sum_{t=0}^{\infty} \beta^t \frac{c_t^{1-\gamma}}{1-\gamma},$$

where $\beta = 0.95$ is the agent's discount factor and $\gamma$ is the coeffiicent of relative risk aversion.

A perfectly competitive representative firm rents capital $k_t$ from the household and produces using the production function $f(k) = A k^{\alpha}$. The aggregate technology in the economy is

$$g_t + c_t + k_{t+1} \le Ak_t^{\alpha} + (1-\delta) k_t,$$

where $g_t$ is government spending, $A$ is a constant productivity factor, and $\delta$ is the depreciation rate of capital.

The consumer faces two taxes: a consumption tax $\tau_{ct}$ and tax on earnings to capital $\tau_{kt}$. The household budget constraint is

$$ \sum_{t=0}^{\infty} q_t \left\{(1 + \tau_{ct}) c_t + [k_{t+1} - (1 - \delta) k_t] \right\} \le \sum_{t=0}^{\infty} q_t \left\{\eta_t k_t -\tau_{kt} (\eta_t - \delta) k_t + w_t\right\}.$$

Here $q_t$ is the time zero price of one unit of consumption in period $t$, $\eta_t$ is the pre-tax price households recieve by lending capital to firms in time $t$, and $w_t$ is the time $t$ wage households earn on the inelastically supplied one unit of labor.

The government faces the budget constraint

$$ \sum_{t=0}^{\infty} q_t g_t \le \sum_{t=0}^{\infty} q_t \left\{\tau_{ct} c_t + \tau_{kt} (\eta_t - \delta) k_t \right\}$$

## Equilibrium conditions

A competitive equilibrium is 

- a budget-feasible government policy $\left\{g_t, \tau_{ct}, \tau_{kt} \right\}_{t=0}^{\infty}$
- a feasible allocation $\left\{c_t, k_{t+1}\right\}_{t=0}^{\infty}$
- and prices $\left\{q_t \right\}_{t=0}^{\infty}$ 

such that

- given prices and the government policy, the allocation solves the household and firm problems.

### Firm optimization

The firm's problem is very simple:

$$\max_{k_t} \sum_{t=0}^{\infty} q_t \left[A k_t^{\alpha} - \eta_t k_t \right].$$

The zero-profit condition is $\eta_t = A \alpha k_t^{\alpha-1}$.

### Household optimization

The problem of the household is to maximize $U$ subject to the budget constraint, taking prices and the government policy as given.

Assuming an interior solution and that the budget constraint holds with equality, the first order necessary condition of the household can be summarized by an Euler equation

$$1 = \beta \left(\frac{c_{t+1}}{c_t}\right)^{-\gamma} \frac{(1+\tau_{ct})}{(1+\tau_{ct+1})} \left[(1 - \tau_{kt+1})(\alpha A k_{t+1}^{\alpha-1} - \delta) + 1 \right]$$

and a law of motion for capital

$$k_{t+1} = A k_{t}^{\alpha} + (1 - \delta) k_t - g_t - c_t.$$

### Prices and government policy

Imposing a no arbitrage condition on the household allows one to derive the following condition on prices (see RMT3 section 11.5.1 for more details):

$$\frac{q_t}{q_{t+1}} = \left[(1 - \tau_{kt+1}(\eta_{t+1} - \delta) + 1 \right].$$

After a normalization that $q_0 = 1$, the above equation pins down the sequence of prices.

In the experiments below we will assume that the goverment policy is exogenously given and solve for how the solutions to the household and firms problems adjust to shocks to the government policy.

### Other equilibrium conditions

There are a few other variables of interest. 
$$\begin{align*}
\eta_t &= \alpha A k_t^{\alpha-1} \\
w_t &= A k_t^{\alpha} - k_t \eta_t \\
\bar{R}_{t+1} &= \frac{(1+\tau_{ct})}{(1+\tau_{ct+1})} \left[(1 - \tau_{kt+1})(\alpha A k_{t+1}^{\alpha-1} - \delta) + 1 \right]
\end{align*}$$

Here $w_t$ is the wage rate the firm must pay to the household. Above we wrote the firm's problem with capital as the only input into production. All the equilibrium conditions above are still correct if we instead assume that the firm operates a constant returns to technology $F(k, n)$ and assume that the household inelastically supplies a single unit of labor and is paid a wage of $w_t$.

$\bar{R}_{t+1}$ is rate at which the price and taxes allow the conusmer to substitute consumption in period $t$ for consumption in period $t+1$.

### Experiments

We will do a number of experiments and analyze the transition path for the equilibrium in each case:

1. A foreseen once-and-for-all increase in $g$ from 0.2 to 0.4 in period 10.
2. A foreseen once-and-for-all increase in $\tau_c$ from 0.0 to 0.2 in period 10.
3. A foreseen once-and-for-all increase in $\tau_k$ from 0.0 to 0.2 in period 10.
3. A foreseen one-time increase in $g$ from 0.2 to 0.4 in period 10, after which $g$ returns to 0.2 forever

## Enter `Dolo.jl`

Let's perform these experiments with [Dolo.jl](https://github.com/EconForge/Dolo.jl).
This will allow us to cleanly separate the model definition from the code
used to exploit the results. General guidelines on how to write models are available
[here](http://www.econforge.org/Dolo.jl/latest/model_specification.html).

Here's the dolo version of the model that we will be using.

In [1]:
#Note: This assumes you are loading a file in the same directory as this .ipynb.
print(open(readstring, "rmt3_ch11.yaml"))

# Model in chapter 11 of Recursive Macroeconomic Theory 3rd edition by
# Ljvinquist and Sargent
name: fiscal_growth

symbols:
    states: [k, tau_c, tau_k]
    controls: [c]
    exogenous: [g, exog_tau_c, exog_tau_k]
    parameters: [beta, gamma, delta, alpha, A]

definitions:
    # Equations from 11.6.8
    eta: alpha*A*k^(alpha-1)
    w: A*k^alpha - k*eta
    Rb: (1+tau_c)/(1+tau_c(1))*((1-tau_k(1)) * (alpha*A*k(1)^(alpha-1) - delta) + 1.)

equations:

    arbitrage:
        # Equation 11.6.3
        - 1 = beta*(c(+1)/c)^(-gamma)*(1+tau_c)/(1+tau_c(1))*((1-tau_k(1))*(eta(1)-delta) + 1) | 0 <= c <= inf

    transition:
        # Equation 11.6.1
        - k = A*k(-1)^alpha+(1-delta)*k(-1)-c(-1)-g
        # We have the states tau_c and tau_k just follow exactly the sequence
        # of shocks that we supply.
        - tau_c = exog_tau_c
        - tau_k = exog_tau_k


calibration:
    # parameters
    beta: 0.95
    gamma: 2.0
    delta: 0.2
    alpha: 0.33
    A: 1.
    exog_tau_c: 0.0

The general formulation of a model in Dolo specifies a controlled process: 

$$s_t = g(m_{t-1}, s_{t-1}, x_{t-1}, m_tt)$$

where $s_t$ is a vector of states, $x_t$ a vector of controls taken at each state, and $m_t$ a vector of exogenous variables. This equation is defined in the `equations:transition` block.

For our model, there is essentially one state $k_t$ and one control $c_t$. Note that in this particular case choosing $c_t$ is equivalent to choosing $k_{t+1}$, but this is not true in general, for instance if there are many control for one state.

Notice in the model file the addition of `tau_c` and `tau_k` as state variables. These dummy states track the innovations `exog_tau_c`, `exog_tau_k`. This was necessary in order to to have `tau_c` in both period `t` and period `t+1` in the Euler equation defined in the block `equations:arbitrage` whose conventional definition is:

$$0 = E_t \left[ f(m_t, s_t, x_t, s_{t+1}, x_{t+1}, m_{t+1}) \right]$$

In this note we are interested in the predictible effect of a preannounced government policy on the decisions by the representative agent, not in a time-invariant decision rule. Hence we will use the `perfect_foresight` solution routine. This module implements the stacked time algorithm for solving determinsitc problems. A description of this algorithm can be found [here](http://www.econforge.org/Dolo.jl/latest/algos.html#Perfect-Foresight-1).

In a perfect foresight simulation, the computational cost of adding additional states is relatively low, because one does not need to solve for the optimal decisions at each point of the state space. For more information on which variables can appear at what times in each type of equation, see the [dolo model classification](http://www.econforge.org/Dolo.jl/latest/model_specification.html) . 

Let's now import the model and display it. The display shows the residuals of each equation, evaluated at the values specified in the `calibration` section of the modfile.

In [2]:
using Plots
gr()
using Dolo
using AxisArrays
model = yaml_import("rmt3_ch11.yaml")
model

Base.Range{T

name,fiscal_growth
filename,rmt3_ch11.yaml
Type,Equation
transition,\[k_{t} = \left(\left(A \left(k_{t-1}\right)^{\alpha}+\left(1-\delta\right) k_{t-1}-c_{t-1}\right)-g_{t}\right)\]
,"\[\tau_{c,t} = exog_{\tau,c,t}\]"
,"\[\tau_{k,t} = exog_{\tau,k,t}\]"
arbitrage,"\[1 = \frac{\beta \left(\frac{c_{t+1}}{c_{t}}\right)^{-\gamma} 1+\tau_{c,t}}{1+\tau_{c,t+1}} \left(1-\tau_{k,t+1}\right) \left(\eta_{t+1}-\delta\right)+1\]"


Now let's construct some dictionaries to hold the shocks in our experiments.

In the `deterministic_solve` function, simulations last for `T` periods. dolo assumes that if a given time-series of shocks is less than `T` in length, that the corresponding shock will hold its last given value until period `T`. Thus, to implement the once-and-for-all increase in our exogenous variables we simply need to pass the values before the increase and a single instance of the value after the increase. 

We do this below.

In [3]:
shocks_1 = Dict(:g => vcat(fill(0.2, 9), 0.4))
shocks_2 = Dict(:exog_tau_c => vcat(fill(0.0, 9), 0.2))
shocks_3 = Dict(:exog_tau_k => vcat(fill(0.0, 9), 0.2))
shocks_4 = Dict(:g => vcat(fill(0.2, 9), [0.4, 0.2]))
no_shocks = Dict(:g => [model.calibration.flat[:g]])

# also specify how long to simulate and plot
T = 101  # simulation length
p_T = 40;  # Periods to plot

} where T) in module Base at deprecated.jl:56 overwritten in module StatsBase at C:\Users\Jesse Perla\.julia\v0.6\StatsBase\src\hist.jl:535.


## Experiment 1  (RMT 3 Figure 11.9.1)

Now we are ready to do the experiment corresponding to the once-and-for-all increase to $g$ in period $10$.

First, we solve for the equilibrium transition in this experiment

In [6]:
sol = perfect_foresight(model, shocks_1, T=T, complementarities=false);

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     1.973629e-02     2.192046e+00
     2     2.673832e-04     2.879876e-01
     3     1.064317e-07     5.991851e-03
     4     1.287859e-14     1.965923e-06


For comparison, we will also want to solve for the equilibrium path assuming no changes to any of the government variables ($g=0.2$, $\tau_c=0$, and $\tau_k=0$ forever)

In [7]:
sol_ref = perfect_foresight(model, no_shocks, T=T, complementarities=false)

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------


2-dimensional AxisArray{Float64,2,...} with axes:
    :V, Symbol[:g, :exog_tau_c, :exog_tau_k, :k, :tau_c, :tau_k, :c, :w, :Rb, :eta]
    :T, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9  …  92, 93, 94, 95, 96, 97, 98, 99, 100, 101]
And data, a 10×102 Array{Float64,2}:
 0.2       0.2       0.2       0.2       …  0.2       0.2       0.2     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 1.48996   1.48996   1.48996   1.48996      1.48996   1.48996   1.48996 
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.642645  0.642645  0.642645  0.642645     0.642645  0.642645  0.642645
 0.764226  0.764226  0.764226  0.764226     0.764226  0.764226  0.764226
 1.05263   1.05263   1.05263   1.05263      1.05263   1.05263   1.05263 
 0.252632  0.252632  0.252632  0.252632     0.252632  0.252632  0.252632

We will also want to show the steady state. We could solve for it numerically,
but since the model is simple enough, the calibrated values given with 
the model, already solve it in closed form.

In [8]:
ss = model.calibration.flat[:k, :c, :eta, :w, :g, :Rb]
kbar, cbar, etabar, wbar, gbar, Rbar = ss;

Now all the computations are done, we will work on constructing figure 11.9.1 in RMT3.

Note that we will first construct the plot by hand, which will require some effort on our part. Then below we will show how to leverage convenience functions in `dolo` to make that task easier.

In [12]:
# Set up plotting materials 
variables = [:k, :c, :Rb, :eta, :g, :w]
marker_color = [:"#E24A33", :black, :"#348ABD"]
line_dash = [:line, :dash, :dashdot];

In [13]:
function plot_irfs(
        sims::Vector{<:AxisArray},
        variables::Vector{Symbol},
        color_style::Array{Any},
        line_style::Array{Symbol},
        titles::Vector=variables,
        T::Int=length(sims[1][Axis{:T}]) 

    )
    n_row_col = Dict(
        1 => (1, 1),
        2 => (1, 2),
        3 => (1, 3),
        4 => (2, 2),
        5 => (2, 3),
        6 => (2, 3),
        7 => (4, 2),
        8 => (4, 2),
        9 => (3, 3),
        10 => (4, 3),
        11 => (4, 3),
        12 => (4, 3)
    )
    n_sim = length(sims)
    n_var = length(variables)
    
    if n_var > 12
        error("can only plot up to 12 variables for you...")
    end
    
    y = Array{Vector}(n_var)
    t = string.(titles)
    titles_string = Array{String}(1, n_var)
    
    p = plot(layout = n_var, legend = :none)
    
    for i in 1:n_sim
        for i_v in 1: n_var
            v = variables[i_v]
            y[i_v] = sims[i][v][1:T]
            titles_string[i_v] = t[i_v]
        end
        plot!(y, layout = n_var, color = color_style[i], line = (line_style[i], 3), title = titles_string, legend=:none)
    end
    
    p
        
end

plot_irfs (generic function with 3 methods)

In [14]:
variables = [:k, :c, :Rb, :eta, :g, :w]
fig_11_9_1 = plot_irfs([sol, sol_ref], variables, marker_color, line_dash, variables, p_T)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.5 
 
 
 1.6 
 
 
 1.7 
 
 
 1.8 
 
 
 1.9 
 
 
 2.0 
 
 
 k 
 
<polyline clip-path="url(#clip7402)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.1651,175.869 42.7843,163.827 46.4035,150.808 50.0228,136.451 53.642,120.381 57.2612,102.217 60.8804,81.5833 64.4997,58.1187 68.1189,31.4961 71.7381,50.7981 
 75.3574,67.3401 78.9766,81.5546 82.5958,93.7991 86.215,104.37 89.8343,113.514 93.4535,121.438 97.0727,128.316 100.692,134.295 104.311,139.498 107.93,144.032 
 111.55,147.987 115.169,151.44 118.788,154.457 122.407,157.094 126.027,159.402 129.646,161.422 133.265,163.192 136.884,164.742 140.503,166.101 144.123,167.293 
 147.742,168.338 151.361,169.255 154.98,170.059 158.6,170.766 162.219,171.386 165.838,171.93 169.457,172.408 173.077,172.828 176.696,173.197 180.315,173.521 
 
 "/>
<polyline clip-path="url(#clip7402)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 39.1651,175.869 42.7843,175.869 46.4035,175.869 50.0228,175.869 53.642,175.869 57.2612,175.869 60.8804,175.869 64.4997,175.869 68.1189,175.869 71.7381,175.869 
 75.3574,175.869 78.9766,175.869 82.5958,175.869 86.215,175.869 89.8343,175.869 93.4535,175.869 97.0727,175.869 100.692,175.869 104.311,175.869 107.93,175.869 
 111.55,175.869 115.169,175.869 118.788,175.869 122.407,175.869 126.027,175.869 129.646,175.869 133.265,175.869 136.884,175.869 140.503,175.869 144.123,175.869 
 147.742,175.869 151.361,175.869 154.98,175.869 158.6,175.869 162.219,175.869 165.838,175.869 169.457,175.869 173.077,175.869 176.696,175.869 180.315,175.869 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.45 
 
 
 0.50 
 
 
 0.55 
 
 
 0.60 
 
 
 c 
 
<polyline clip-path="url(#clip7403)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 239.165,67.0105 242.784,68.1198 246.404,70.3612 250.023,73.7692 253.642,78.3827 257.261,84.2383 260.88,91.3621 264.5,99.7614 268.119,109.418 271.738,117.841 
 275.357,125.194 278.977,131.616 282.596,137.23 286.215,142.139 289.834,146.434 293.453,150.194 297.073,153.488 300.692,156.373 304.311,158.902 307.93,161.12 
 311.55,163.064 315.169,164.77 318.788,166.267 322.407,167.581 326.027,168.734 329.646,169.747 333.265,170.636 336.884,171.416 340.503,172.102 344.123,172.704 
 347.742,173.233 351.361,173.698 354.98,174.107 358.6,174.466 362.219,174.781 365.838,175.058 369.457,175.301 373.077,175.515 376.696,175.703 380.315,175.869 
 
 "/>
<polyline clip-path="url(#clip7403)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 239.165,31.4961 242.784,31.4961 246.404,31.4961 250.023,31.4961 253.642,31.4961 257.261,31.4961 260.88,31.4961 264.5,31.4961 268.119,31.4961 271.738,31.4961 
 275.357,31.4961 278.977,31.4961 282.596,31.4961 286.215,31.4961 289.834,31.4961 293.453,31.4961 297.073,31.4961 300.692,31.4961 304.311,31.4961 307.93,31.4961 
 311.55,31.4961 315.169,31.4961 318.788,31.4961 322.407,31.4961 326.027,31.4961 329.646,31.4961 333.265,31.4961 336.884,31.4961 340.503,31.4961 344.123,31.4961 
 347.742,31.4961 351.361,31.4961 354.98,31.4961 358.6,31.4961 362.219,31.4961 365.838,31.4961 369.457,31.4961 373.077,31.4961 376.696,31.4961 380.315,31.4961 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.01 
 
 
 1.02 
 
 
 1.03 
 
 
 1.04 
 
 
 1.05 
 
 
 Rb 
 
<polyline clip-path="url(#clip7404)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 439.165,46.9908 442.784,62.8456 446.404,79.3432 450.023,96.6934 453.642,115.025 457.261,134.379 460.88,154.706 464.5,175.869 468.119,160.715 471.738,146.917 
 475.357,134.405 478.977,123.1 482.596,112.919 486.215,103.776 489.834,95.5857 493.453,88.2659 497.073,81.7369 500.692,75.9233

## Change in IES (RMT3 Figure 11.9.2)

We now want to highlight the impact of the parameter $\gamma$ in the household's preferences. The intertemporal elasticity of substitution (IES) for these preferences is given by $1/\gamma$. This means that higher values of $\gamma$ decrease IES, making the household less willing to substitute consumption across time.

Let's perform the same experiment as above, but this time with $\gamma$ set to $0.2$ instead of $2.0$.

In [15]:
# Change gamma and compute solution again
set_calibration!(model, gamma=0.2)
sol_ies = perfect_foresight(model, shocks_1, T=T, complementarities=false);

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     4.365597e-03     1.937257e+00
     2     2.997518e-05     4.629510e-02
     3     1.681675e-09     3.678048e-04


In [16]:
variables = [:k, :c, :Rb, :eta, :g, :w]
fig_11_9_2 = plot_irfs([sol, sol_ref, sol_ies], variables, marker_color, line_dash, variables, p_T)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.5 
 
 
 1.6 
 
 
 1.7 
 
 
 1.8 
 
 
 1.9 
 
 
 2.0 
 
 
 k 
 
<polyline clip-path="url(#clip7602)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.1651,175.869 42.7843,163.827 46.4035,150.808 50.0228,136.451 53.642,120.381 57.2612,102.217 60.8804,81.5833 64.4997,58.1187 68.1189,31.4961 71.7381,50.7981 
 75.3574,67.3401 78.9766,81.5546 82.5958,93.7991 86.215,104.37 89.8343,113.514 93.4535,121.438 97.0727,128.316 100.692,134.295 104.311,139.498 107.93,144.032 
 111.55,147.987 115.169,151.44 118.788,154.457 122.407,157.094 126.027,159.402 129.646,161.422 133.265,163.192 136.884,164.742 140.503,166.101 144.123,167.293 
 147.742,168.338 151.361,169.255 154.98,170.059 158.6,170.766 162.219,171.386 165.838,171.93 169.457,172.408 173.077,172.828 176.696,173.197 180.315,173.521 
 
 "/>
<polyline clip-path="url(#clip7602)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 39.1651,175.869 42.7843,175.869 46.4035,175.869 50.0228,175.869 53.642,175.869 57.2612,175.869 60.8804,175.869 64.4997,175.869 68.1189,175.869 71.7381,175.869 
 75.3574,175.869 78.9766,175.869 82.5958,175.869 86.215,175.869 89.8343,175.869 93.4535,175.869 97.0727,175.869 100.692,175.869 104.311,175.869 107.93,175.869 
 111.55,175.869 115.169,175.869 118.788,175.869 122.407,175.869 126.027,175.869 129.646,175.869 133.265,175.869 136.884,175.869 140.503,175.869 144.123,175.869 
 147.742,175.869 151.361,175.869 154.98,175.869 158.6,175.869 162.219,175.869 165.838,175.869 169.457,175.869 173.077,175.869 176.696,175.869 180.315,175.869 
 
 "/>
<polyline clip-path="url(#clip7602)" style="stroke:#348abd; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 6, 2, 6" points="
 39.1651,175.869 42.7843,175.368 46.4035,174.669 50.0228,173.52 53.642,171.508 57.2612,167.92 60.8804,161.52 64.4997,150.235 68.1189,130.778 71.7381,147.831 
 75.3574,158.293 78.9766,164.794 82.5958,168.868 86.215,171.434 89.8343,173.056 93.4535,174.083 97.0727,174.734 100.692,175.148 104.311,175.411 107.93,175.578 
 111.55,175.684 115.169,175.751 118.788,175.794 122.407,175.821 126.027,175.838 129.646,175.849 133.265,175.856 136.884,175.861 140.503,175.864 144.123,175.866 
 147.742,175.867 151.361,175.867 154.98,175.868 158.6,175.868 162.219,175.868 165.838,175.868 169.457,175.869 173.077,175.869 176.696,175.869 180.315,175.869 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.45 
 
 
 0.50 
 
 
 0.55 
 
 
 0.60 
 
 
 c 
 
<polyline clip-path="url(#clip7603)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 239.165,66.7178 242.784,67.8179 246.404,70.0409 250.023,73.4207 253.642,77.9962 257.261,83.8036 260.88,90.8686 264.5,99.1986 268.119,108.776 271.738,117.13 
 275.357,124.422 278.977,130.791 282.596,136.358 286.215,141.227 289.834,145.487 293.453,149.216 297.073,152.482 300.692,155.344 304.311,157.852 307.93,160.051 
 311.55,161.98 315.169,163.672 318.788,165.156 322.407,166.459 326.027,167.603 329.646,168.607 333.265,169.489 336.884,170.263 340.503,170.943 344.123,171.54 
 347.742,172.065 351.361,172.526 354.98,172.931 358.6,173.287 362.219,173.6 365.838,173.874 369.457,174.116 373.077,174.328 376.696,174.515 380.315,174.679 
 
 "/>
<polyline clip-path="url(#clip7603)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 239.165,31.4961 242.784,31.4961 246.404,31.4961 250.023,31.4961 253.642,31.4961 257.261,31.4961 260.88,31.4961 264.5,31.4961 268.119,31.4961 271.738,31.4961 
 275.357,31.4961 278.977,31.4961 282.596,31.4961 286.215,31.4961 289.834,31.4961 293.453,31.4961 297.073,31.4961 300.692,31.4961 304.311,31.4961 307.93,31.4961 
 311.55,31.4961 315.169,31.4961 318.788,31.4961 322.407,31.4961 326.027,31.4961 329.646,31

In the figure above the solid red line represents the experiment when $\gamma=2.0$, the dash-dotted blue line tracks the equilibrium in the experiment when $\gamma=0.2$, and the black dashed line tracks the steady state.

Notice that because the household is more willing to move consumption across time when $\gamma$ is smaller, the movement from initial to final levels of consumption is both larger and quicker. This lets the consumption stream "swallow" most of the impact of the increase in $g$ and capital doesn't respond as much (think about the household budget constraint). Factor prices and $\bar{R}$ are functions of the captial stock, so they also do not respond as sharply.

## Consumption tax shocks (RMT 3 Figure 11.9.4)

Now let's consider the second experiment: a once-and-for-all increase in $\tau_c$ from 0 to 0.2 in period 10.

In [17]:
# reset gamma
set_calibration!(model, gamma=2.0)

sol2 = perfect_foresight(model, shocks_2, T=T, complementarities=false)

set_calibration!(model, gamma=0.2)

sol2_ies = perfect_foresight(model, shocks_2, T=T, complementarities=false)

variables=[:k, :c, :Rb, :eta, :tau_c]
fig_11_9_4 = plot_irfs([sol2, sol_ref, sol2_ies], variables, marker_color, line_dash, variables, p_T)

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     2.048037e-02     1.965409e+00
     2     2.372421e-04     4.325968e-02
     3     3.251771e-08     5.450922e-04
     4     8.881784e-16     7.802051e-08
Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     4.976031e-02     2.096298e+00
     2     3.567140e-03     1.512156e-01
     3     1.885802e-05     1.069803e-02
     4     5.363949e-10     5.290224e-05


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.1 
 
 
 1.2 
 
 
 1.3 
 
 
 1.4 
 
 
 k 
 
<polyline clip-path="url(#clip7802)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.1651,31.4961 42.7843,34.3525 46.4035,37.4614 50.0228,40.95 53.642,44.9686 57.2612,49.6986 60.8804,55.3638 64.4997,62.2447 68.1189,70.6994 71.7381,81.192 
 75.3574,74.0242 78.9766,67.8684 82.5958,62.5878 86.215,58.0625 89.8343,54.1876 93.4535,50.8719 97.0727,48.0363 100.692,45.6126 104.311,43.5418 107.93,41.7731 
 111.55,40.2629 115.169,38.9738 118.788,37.8736 122.407,36.9348 126.027,36.1339 129.646,35.4507 133.265,34.8679 136.884,34.3709 140.503,33.9471 144.123,33.5857 
 147.742,33.2775 151.361,33.0147 154.98,32.7907 158.6,32.5997 162.219,32.4369 165.838,32.298 169.457,32.1797 173.077,32.0788 176.696,31.9928 180.315,31.9195 
 
 "/>
<polyline clip-path="url(#clip7802)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 39.1651,31.4961 42.7843,31.4961 46.4035,31.4961 50.0228,31.4961 53.642,31.4961 57.2612,31.4961 60.8804,31.4961 64.4997,31.4961 68.1189,31.4961 71.7381,31.4961 
 75.3574,31.4961 78.9766,31.4961 82.5958,31.4961 86.215,31.4961 89.8343,31.4961 93.4535,31.4961 97.0727,31.4961 100.692,31.4961 104.311,31.4961 107.93,31.4961 
 111.55,31.4961 115.169,31.4961 118.788,31.4961 122.407,31.4961 126.027,31.4961 129.646,31.4961 133.265,31.4961 136.884,31.4961 140.503,31.4961 144.123,31.4961 
 147.742,31.4961 151.361,31.4961 154.98,31.4961 158.6,31.4961 162.219,31.4961 165.838,31.4961 169.457,31.4961 173.077,31.4961 176.696,31.4961 180.315,31.4961 
 
 "/>
<polyline clip-path="url(#clip7802)" style="stroke:#348abd; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 6, 2, 6" points="
 39.1651,31.4961 42.7843,32.1811 46.4035,33.1409 50.0228,34.7265 53.642,37.5328 57.2612,42.6383 60.8804,52.0746 64.4997,69.8345 68.1189,104.333 71.7381,175.869 
 75.3574,119.521 78.9766,83.9537 82.5958,62.3521 86.215,49.5102 89.8343,41.9676 93.4535,37.568 97.0727,35.0119 100.692,33.5301 104.311,32.6723 107.93,32.176 
 111.55,31.8891 115.169,31.7232 118.788,31.6273 122.407,31.5719 126.027,31.5399 129.646,31.5214 133.265,31.5107 136.884,31.5045 140.503,31.501 144.123,31.4989 
 147.742,31.4977 151.361,31.497 154.98,31.4966 158.6,31.4964 162.219,31.4962 165.838,31.4962 169.457,31.4961 173.077,31.4961 176.696,31.4961 180.315,31.4961 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.5 
 
 
 0.6 
 
 
 0.7 
 
 
 0.8 
 
 
 c 
 
<polyline clip-path="url(#clip7803)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 239.165,99.9503 242.784,99.8424 246.404,99.6157 250.023,99.2543 253.642,98.7352 257.261,98.027 260.88,97.0873 264.5,95.8584 268.119,94.2618 271.738,113.948 
 275.357,112.335 278.977,110.959 282.596,109.786 286.215,108.787 289.834,107.934 293.453,107.207 297.073,106.588 300.692,106.06 304.311,105.61 307.93,105.226 
 311.55,104.899 315.169,104.62 318.788,104.383 322.407,104.18 326.027,104.007 329.646,103.86 333.265,103.735 336.884,103.628 340.503,103.537 344.123,103.459 
 347.742,103.393 351.361,103.336 354.98,103.288 358.6,103.247 362.219,103.212 365.838,103.182 369.457,103.157 373.077,103.135 376.696,103.117 380.315,103.101 
 
 "/>
<polyline clip-path="url(#clip7803)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 239.165,103.01 242.784,103.01 246.404,103.01 250.023,103.01 253.642,103.01 257.261,103.01 260.88,103.01 264.5,103.01 268.119,103.01 271.738,103.01 
 275.357,103.01 278.977,103.01 282.596,103.01 286.215,103.01 289.834,103.01 293.453,103.01 297.073,103.01 300.692,103.01 304.311,103.01 307.93,103.01 
 311.55,103.01 315.169,103.01 318.788,103.01 322.407,103.01 326.027,103.01 329.646,103.01 333.265,103.01 336.884,103.01 340.503,103.01 344.123,10

## Capital tax shocks (RMT 3 Figure 11.9.5)

Next, we turn to the third experiment: a once-and-for-all increase in $\tau_k$ from 0 to 0.2 in period 10

In [21]:
sol3 = perfect_foresight(model, shocks_3, T=T, complementarities=false)

# here we also want to look at the impact of changing the ies
set_calibration!(model, gamma=0.2)
sol3_ies = perfect_foresight(model, shocks_3, T=T, complementarities=false)

set_calibration!(model, gamma=2.0)  # reset gamma

variables = [:k, :c, :Rb, :eta, :tau_k]
fig_11_9_5 = plot_irfs([sol3, sol_ref, sol3_ies], variables, marker_color, line_dash, variables, p_T)

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     1.560191e-03     2.114157e+00
     2     1.677112e-05     1.471815e-01
     3     1.569917e-09     1.373012e-03
Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     1.560191e-03     2.123515e+00
     2     1.677118e-05     1.551257e-01
     3     1.569939e-09     1.471872e-03


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.40 
 
 
 1.42 
 
 
 1.44 
 
 
 1.46 
 
 
 1.48 
 
 
 k 
 
<polyline clip-path="url(#clip8402)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 52.5462,31.4961 55.9939,35.1731 59.4416,39.1725 62.8893,43.6526 66.3369,48.798 69.7846,54.8281 73.2323,62.0077 76.68,70.6607 80.1276,81.1865 83.5753,94.0826 
 87.023,105.205 90.4707,114.802 93.9183,123.085 97.366,130.238 100.814,136.416 104.261,141.753 107.709,146.365 111.157,150.352 114.604,153.798 118.052,156.777 
 121.5,159.353 124.947,161.581 128.395,163.508 131.843,165.174 135.29,166.616 138.738,167.863 142.186,168.942 145.633,169.875 149.081,170.682 152.529,171.381 
 155.976,171.985 159.424,172.508 162.872,172.961 166.319,173.352 169.767,173.691 173.215,173.984 176.663,174.238 180.11,174.458 183.558,174.648 187.006,174.812 
 
 "/>
<polyline clip-path="url(#clip8402)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 52.5462,31.4961 55.9939,31.4961 59.4416,31.4961 62.8893,31.4961 66.3369,31.4961 69.7846,31.4961 73.2323,31.4961 76.68,31.4961 80.1276,31.4961 83.5753,31.4961 
 87.023,31.4961 90.4707,31.4961 93.9183,31.4961 97.366,31.4961 100.814,31.4961 104.261,31.4961 107.709,31.4961 111.157,31.4961 114.604,31.4961 118.052,31.4961 
 121.5,31.4961 124.947,31.4961 128.395,31.4961 131.843,31.4961 135.29,31.4961 138.738,31.4961 142.186,31.4961 145.633,31.4961 149.081,31.4961 152.529,31.4961 
 155.976,31.4961 159.424,31.4961 162.872,31.4961 166.319,31.4961 169.767,31.4961 173.215,31.4961 176.663,31.4961 180.11,31.4961 183.558,31.4961 187.006,31.4961 
 
 "/>
<polyline clip-path="url(#clip8402)" style="stroke:#348abd; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 6, 2, 6" points="
 52.5462,31.4961 55.9939,31.969 59.4416,32.631 62.8893,33.7221 66.3369,35.6446 69.7846,39.114 73.2323,45.4314 76.68,56.9931 80.1276,78.2725 83.5753,117.805 
 87.023,141.221 90.4707,155.157 93.9183,163.474 97.366,168.447 100.814,171.422 104.261,173.205 107.709,174.272 111.157,174.912 114.604,175.295 118.052,175.525 
 121.5,175.663 124.947,175.745 128.395,175.795 131.843,175.824 135.29,175.842 138.738,175.853 142.186,175.859 145.633,175.863 149.081,175.865 152.529,175.867 
 155.976,175.867 159.424,175.868 162.872,175.868 166.319,175.868 169.767,175.869 173.215,175.869 176.663,175.869 180.11,175.869 183.558,175.869 187.006,175.869 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.64 
 
 
 0.65 
 
 
 0.66 
 
 
 0.67 
 
 
 c 
 
<polyline clip-path="url(#clip8403)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 245.856,137.144 249.303,136.737 252.751,135.886 256.199,134.536 259.646,132.609 263.094,130.002 266.542,126.579 269.989,122.165 273.437,116.532 276.885,124.515 
 280.332,131.424 283.78,137.404 287.228,142.58 290.675,147.06 294.123,150.936 297.571,154.292 301.018,157.196 304.466,159.709 307.914,161.884 311.361,163.766 
 314.809,165.395 318.257,166.805 321.704,168.025 325.152,169.08 328.6,169.994 332.048,170.785 335.495,171.469 338.943,172.061 342.391,172.574 345.838,173.017 
 349.286,173.401 352.734,173.733 356.181,174.021 359.629,174.269 363.077,174.485 366.524,174.671 369.972,174.832 373.42,174.972 376.867,175.092 380.315,175.197 
 
 "/>
<polyline clip-path="url(#clip8403)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 245.856,148.81 249.303,148.81 252.751,148.81 256.199,148.81 259.646,148.81 263.094,148.81 266.542,148.81 269.989,148.81 273.437,148.81 276.885,148.81 
 280.332,148.81 283.78,148.81 287.228,148.81 290.675,148.81 294.123,148.81 297.571,148.81 301.018,148.81 304.466,148.81 307.914,148.81 311.361,148.81 
 314.809,148.81 318.257,148.81 321.704,148.81 325.152,148.81 328.6,148.81 332.048,148.81 335.495,148.81 338.943

## Impulse shock to g (RMT 3 Figure 11.9.6)

Finally, we turn to the fourth experiment: a one-time shock to $g$ from 0.2 to 0.4 in period 10, then back to 0.2 forever.

In [22]:
sol4 = perfect_foresight(model, shocks_4, T=T, complementarities=false)

variables = [:k, :c, :Rb, :eta, :g]
fig_11_9_6 = plot_irfs([sol4, sol_ref], variables, marker_color, line_dash, variables, p_T)

Iter     f(x) inf-norm    Step 2-norm 
------   --------------   --------------
     0     2.000000e-01              NaN
     1     7.481441e-04     2.196956e-01
     2     3.695996e-07     4.228965e-03
     3     7.527312e-14     1.829582e-06


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.425 
 
 
 1.450 
 
 
 1.475 
 
 
 1.500 
 
 
 1.525 
 
 
 1.550 
 
 
 1.575 
 
 
 k 
 
<polyline clip-path="url(#clip8602)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 45.8557,110.156 49.3033,104.177 52.751,97.6811 56.1987,90.4279 59.6464,82.1421 63.094,72.5062 66.5417,61.1516 69.9894,47.6485 73.4371,31.4961 76.8847,175.869 
 80.3324,166.29 83.7801,158.092 87.2278,151.079 90.6754,145.085 94.1231,139.963 97.5708,135.587 101.018,131.851 104.466,128.661 107.914,125.939 111.361,123.616 
 114.809,121.634 118.257,119.943 121.704,118.501 125.152,117.271 128.6,116.222 132.048,115.328 135.495,114.565 138.943,113.915 142.391,113.36 145.838,112.887 
 149.286,112.484 152.734,112.141 156.181,111.848 159.629,111.598 163.077,111.385 166.524,111.204 169.972,111.049 173.42,110.917 176.867,110.805 180.315,110.709 
 
 "/>
<polyline clip-path="url(#clip8602)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 45.8557,110.156 49.3033,110.156 52.751,110.156 56.1987,110.156 59.6464,110.156 63.094,110.156 66.5417,110.156 69.9894,110.156 73.4371,110.156 76.8847,110.156 
 80.3324,110.156 83.7801,110.156 87.2278,110.156 90.6754,110.156 94.1231,110.156 97.5708,110.156 101.018,110.156 104.466,110.156 107.914,110.156 111.361,110.156 
 114.809,110.156 118.257,110.156 121.704,110.156 125.152,110.156 128.6,110.156 132.048,110.156 135.495,110.156 138.943,110.156 142.391,110.156 145.838,110.156 
 149.286,110.156 152.734,110.156 156.181,110.156 159.629,110.156 163.077,110.156 166.524,110.156 169.972,110.156 173.42,110.156 176.867,110.156 180.315,110.156 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 0.625 
 
 
 0.630 
 
 
 0.635 
 
 
 0.640 
 
 
 c 
 
<polyline clip-path="url(#clip8603)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 245.856,86.6304 249.303,88.5133 252.751,92.4236 256.199,98.5738 259.646,107.25 263.094,118.818 266.542,133.731 269.989,152.534 273.437,175.869 276.885,154.566 
 280.332,136.405 283.78,120.923 287.228,107.726 290.675,96.4752 294.123,86.8849 297.571,78.7098 301.018,71.7412 304.466,65.8011 307.914,60.7376 311.361,56.4214 
 314.809,52.7423 318.257,49.6062 321.704,46.933 325.152,44.6544 328.6,42.7121 332.048,41.0565 335.495,39.6453 338.943,38.4424 342.391,37.417 345.838,36.543 
 349.286,35.798 352.734,35.163 356.181,34.6217 359.629,34.1603 363.077,33.767 366.524,33.4318 369.972,33.1461 373.42,32.9025 376.867,32.6949 380.315,32.5179 
 
 "/>
<polyline clip-path="url(#clip8603)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" stroke-dasharray="12, 8" points="
 245.856,31.4961 249.303,31.4961 252.751,31.4961 256.199,31.4961 259.646,31.4961 263.094,31.4961 266.542,31.4961 269.989,31.4961 273.437,31.4961 276.885,31.4961 
 280.332,31.4961 283.78,31.4961 287.228,31.4961 290.675,31.4961 294.123,31.4961 297.571,31.4961 301.018,31.4961 304.466,31.4961 307.914,31.4961 311.361,31.4961 
 314.809,31.4961 318.257,31.4961 321.704,31.4961 325.152,31.4961 328.6,31.4961 332.048,31.4961 335.495,31.4961 338.943,31.4961 342.391,31.4961 345.838,31.4961 
 349.286,31.4961 352.734,31.4961 356.181,31.4961 359.629,31.4961 363.077,31.4961 366.524,31.4961 369.972,31.4961 373.42,31.4961 376.867,31.4961 380.315,31.4961 
 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 1.045 
 
 
 1.050 
 
 
 1.055 
 
 
 1.060 
 
 
 Rb 
 
<polyline clip-path="url(#clip8604)" style="stroke:#e24933; stroke-width:3; stroke-opacity:1; fill:none" points="
 445.856,106.738 449.303,113.193 452.751,120.333 456.199,128.406 459.646,137.681 463.094,148.461 466.542,161.072 469.989,175.869 473.437,31.4961 476.885,41.9963 
 480.332,50.8689 483.78,58.3761 487.228,64.7349 490.675,70.1262 494.123,74.7008 497.571,78.58